In [1]:
import numpy as np
from tensorflow import keras

# Model / data parameters
num_classes = 10
input_shape = (28, 28, 1)

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

# Scale images to the [0, 1] range
x_train = x_train.astype("float32") / 255
x_test = x_test.astype("float32") / 255

# Make sure images have shape (28, 28, 1)
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)
print("x_train shape:", x_train.shape)
print(x_train.shape[0], "train samples")
print(x_test.shape[0], "test samples")

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


In [2]:
from classifier_net import ClassifierNet
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers
import random

test_classifier = ClassifierNet(input_shape=input_shape, num_classes=num_classes)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [3]:
#test_classifier.train(x_train, y_train, epochs=1)

In [22]:
test_classifier.train(x_train, y_train, epochs=1)
score = test_classifier.evaluate(x_test, y_test)

print("Test loss:", score[0])
print("Test accuracy:", score[1])

Train on 54000 samples, validate on 6000 samples
54000/54000 [==============================] - 2s 45us/sample - loss: 0.8156 - acc: 0.7526 - val_loss: 0.3116 - val_acc: 0.9128
Test loss: 0.36095211787223813
Test accuracy: 0.8998


In [23]:
test_weights = test_classifier.get_lstm_weights()
print(test_weights.shape)
test_classifier.load_from_lstm_prediction(test_weights)
#print(test_classifier.get_training_data()['loss'])

(3, 420)


In [24]:
score = test_classifier.evaluate(x_test, y_test)

print("Test loss:", score[0])
print("Test accuracy:", score[1])

Test loss: 0.36095211787223813
Test accuracy: 0.8998


In [30]:
# [batch, timesteps, feature]
from tensorflow.keras import backend
test_weights = test_weights.reshape(1, 3, 420)
lstm = layers.LSTM(420, return_sequences=True)
output = lstm(test_weights)
print(backend.eval(output))

[[[-0.01248909  0.04974105 -0.03434295 ... -0.00075313 -0.00602379
    0.01706288]
  [-0.04520131  0.05300964 -0.02483081 ...  0.01599254  0.00572861
    0.00413595]
  [-0.05248566  0.0842592  -0.0465044  ... -0.00665576 -0.00879703
    0.0207903 ]]]


In [26]:

lstm_model = keras.Sequential()
lstm_model.add(keras.Input(shape=420))

lstm_model.add(layers.LSTM(256, return_sequences=True))

# The output of SimpleRNN will be a 2D tensor of shape (batch_size, 128)
lstm_model.add(layers.SimpleRNN(128))

lstm_model.add(layers.Dense(10))

lstm_model.summary()

ValueError: Input 0 of layer lstm_10 is incompatible with the layer: expected ndim=3, found ndim=2. Full shape received: [None, 420]